# 实验一

## 最大子数组问题

分别实现 4.1 节所讲的求最大子数组问题的分治算法 FIND-MAXIMUM-SUBARRAY 和习题 4.1-5 所提示的非递归线性时间算法

> **最大子数组**：一个数组中和最大的非空连续子数组。

### 1. 分治算法 FIND-MAXIMUM-SUBARRAY

求解 A[low, high] 的最大子数组：

- 求解思路：
    1. 找到 A 的 mid
    2. 求解 A[low..mid] 和 A[mid+1..high] 的最大子数组
    3. 找到跨越中点的最大子数组
    4. 取和最大者
- A[low..high] 和任何 A[i..j] 之间的三种情况
    - low ≤ i ≤ j ≤ mid
    - mid < i ≤ j ≤ high
    - low ≤ i ≤ mid < j ≤ high

In [4]:
import time

**1. 分别实现两个算法的功能函数**

- 分治算法

In [9]:
# 跨越中点的最大子数组
def FIND_MAX_CROSSSING_SUBARRAY(A, low, mid, high):
    left_sum = float('-inf')
    sum = 0
    max_left = mid  # 初始化max_left
    for i in range(mid, low-1, -1):
        sum += A[i]
        if sum > left_sum:
            left_sum = sum
            max_left = i
    right_sum = float('-inf')
    sum = 0
    max_right = mid + 1  # 初始化max_right
    for j in range(mid+1, high+1):
        sum += A[j]
        if sum > right_sum:
            right_sum = sum
            max_right = j
    return (max_left, max_right, left_sum + right_sum)

# 找最大子数组
def FIND_MAX_SUBARRAY(A, low, high):
    if high == low:
        return (low, high, A[low])
    else:
        mid = (low + high) // 2  # 使用整除
        (left_low, left_high, left_sum) = FIND_MAX_SUBARRAY(A, low, mid)
        (right_low, right_high, right_sum) = FIND_MAX_SUBARRAY(A, mid+1, high)
        (cross_low, cross_high, cross_sum) = FIND_MAX_CROSSSING_SUBARRAY(A, low, mid, high)
        if left_sum >= right_sum and left_sum >= cross_sum:
            return (left_low, left_high, left_sum)
        elif right_sum >= left_sum and right_sum >= cross_sum:
            return (right_low, right_high, right_sum)
        else:
            return (cross_low, cross_high, cross_sum)

- 非递归线性时间算法

In [10]:
def FIND_MAX_SUBARRAY_LINEAR(A):
    max_sum = float('-inf') 
    current_sum = 0 
    start = 0 
    max_start = 0 
    max_end = 0 

    for i in range(len(A)):
        current_sum += A[i]
        if current_sum > max_sum:
            max_sum = current_sum
            max_start = start
            max_end = i
        if current_sum < 0:
            current_sum = 0
            start = i + 1

    return (max_start, max_end, max_sum)

**2. 集成在程序中**

In [5]:
def menu():
    print("请选择要使用的算法：")
    print("1. 分治法")
    print("2. 线性法")
    print("0. 退出")
    choice = input("请选择要使用的算法：")
    return choice

**3. 读取文件并输出** 

In [6]:
def read_dataset_from_file(file_path):
    with open(file_path, 'r') as file:
        data = list(map(int, file.read().strip().split()))
    n = data[0]
    if n == 0:
        return []  # 如果 n 为 0，返回空数组
    return data[1:]  # 返回后续的 n 个整数

def main_with_menu(file_path):
    A = read_dataset_from_file(file_path)
    
    if not A:
        print("数据集为空或无效。")
        return

    n = len(A)
    
    while True:
        choice = menu()

        if choice == '1':
            # 使用分治法
            start_time = time.time()
            low, high, max_sum = FIND_MAX_SUBARRAY(A, 0, n-1)
            end_time = time.time()
            if n == 0:
                print(0)
            else:
                print(f"{max_sum} {low} {high} {n}")

        elif choice == '2':
            # 使用线性法
            start_time = time.time()
            low, high, max_sum = FIND_MAX_SUBARRAY_LINEAR(A)
            end_time = time.time()
            if n == 0:
                print(0)
            else:
                print(f"{max_sum} {low} {high} {n}")

        elif choice == '0':
            print("程序已退出。")
            break
        
        else:
            print("无效的选择，请重新输入。")

In [ ]:
if __name__ == "__main__":
    main_with_menu("C:/Users/LENOVO/OneDrive/Documents/CS_Experiments/algorithm_learning/data/dataset_1.txt")
    main_with_menu("C:/Users/LENOVO/OneDrive/Documents/CS_Experiments/algorithm_learning/data/dataset_2.txt")